<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# LLM

## Input

### Import libraries

In [ ]:
import openai
import requests
import json
import os
import time

### Setup variables

In [ ]:
jup_token = os.environ.get('JUPYTERHUB_API_TOKEN')

## Model

### Get Naas Long lived token

In [ ]:
def get_long_lived_token(jup_token):
    url = f"https://auth.naas.ai/bearer/jupyterhub/longlived?token={jup_token}"
    res = requests.get(url)
    res.raise_for_status()
    long_lived_token = res.json()['access_token']
    return long_lived_token

### Create Naas Chat completion

In [ ]:
def create_naas_chat_completion(
    access_token,
    model_id="507dbbc5-88a1-4bd7-8c35-28cea3faaf1f",
    prompt="",
    message="",
):
    # Requests
    url = f"https://api.naas.ai/model/{model_id}/completion"
    payload = json.dumps(
        {
            "id": model_id,
            "payload": json.dumps(
                {
                    "messages": 
                    [
                        {
                            "role": "system",
                            "content": prompt
                        },
                        {
                            "role": "user",
                            "content": message
                        }
                    ]
                }
            )
        }
    )
    headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {access_token}'
    }
    result = None
    retry = 0
    while True:
        try:
            response = requests.post(url, headers=headers, data=payload)
            result = response.json()['completion']['completions'][0]
            break
        except Exception as e:
            print(e)
        retry += 1
        time.sleep(10)
        if retry >= 3:
            break
    return result

### Create OpenAI completion

In [ ]:
def create_openai_completion(
    openai_api_key,
    prompt,
    message
):
    # Init
    openai.api_key = openai_api_key
    
    # Get response
    response = openai.ChatCompletion.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": message
            }
        ]
    )
    return response['choices'][0]['message']['content']

## Output